# pip & import

In [ ]:
!pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 2.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import category_encoders as ce

In [ ]:
apply_train_df = pd.read_csv('apply_train.csv')
resume_train_df = pd.read_csv('resume.csv')
certificate_df = pd.read_csv('resume_certificate.csv')
education_train_df = pd.read_csv('resume_education.csv')
language_train_df = pd.read_csv('resume_language.csv')
company_train_df = pd.read_csv('company.csv')
recruitment_train_df = pd.read_csv('recruitment.csv')

# 함수 정의 & 글로벌 변수

In [ ]:
def recall5(answer_df, submission_df):

    primary_col = answer_df.columns[0]
    secondary_col = answer_df.columns[1]

    # submission의 예측이 각각 5개인지 확인
    prediction_counts = submission_df.groupby(primary_col).size()
    if not all(prediction_counts == 5):
        raise ValueError(f"Each {primary_col} should have exactly 5 {secondary_col} predictions.")


    # submission의 예측된 값들에 null값이 있는지 확인
    if submission_df[secondary_col].isnull().any():
        raise ValueError(f"Predicted {secondary_col} contains NULL values.")

    # 예측값에 중복이 있는지 확인
    duplicated_preds = submission_df.groupby(primary_col).apply(lambda x: x[secondary_col].duplicated().any())
    if duplicated_preds.any():
        raise ValueError(f"Predicted {secondary_col} contains duplicates for some {primary_col}.")


    # Filter the submission dataframe based on the primary_col present in the answer dataframe
    # primary_col 즉 resume_seq가 양측에 있는지 확인 후 남김
    submission_df = submission_df[submission_df[primary_col].isin(answer_df[primary_col])]

    # For each primary_col, get the top 5 predicted secondary_col values
    #
    top_5_preds = submission_df.groupby(primary_col).apply(lambda x: x[secondary_col].head(5).tolist()).to_dict()

    # Convert the answer_df to a dictionary for easier lookup
    true_dict = answer_df.groupby(primary_col).apply(lambda x: x[secondary_col].tolist()).to_dict()


    individual_recalls = []
    for key, val in true_dict.items():
        if key in top_5_preds:
            correct_matches = len(set(true_dict[key]) & set(top_5_preds[key]))
            individual_recall = correct_matches / min(len(val), 5) # 공정한 평가를 가능하게 위하여 분모(k)를 'min(len(val), 5)' 로 설정함
            individual_recalls.append(individual_recall)


    recall = np.mean(individual_recalls)
    return recall

In [ ]:
def minimum_condition(resume, company):
  if(resume['degree'] < company['education']):
    return False
  else:
    return True
   # if(resume['career_month']>company['career_end'] or resume['career_month']<company['career_start']): 다 0임
     # return False
   # else:

In [ ]:
'''def check_salary(salary):
  if salary>3000:
    return 3
  elif salary>2000:
    return 2
  elif salary>1000:
    return 1
  else:
    return 0'''

In [ ]:
'''def check_career_month(career_month):
  if career_month>120:
    return 3
  elif career_month>60:
    return 2
  elif career_month>0:
    return 1
  else:
    return 0'''

In [ ]:
def check_univ_score(univ_score):
  if univ_score>=90:
    return 90
  elif univ_score>=80:
    return 80
  elif univ_score>=70:
    return 70
  elif univ_score>=60:
    return 60
  elif univ_score>=50:
    return 50
  elif univ_score>=40:
    return 40
  elif univ_score>=30:
    return 30
  elif univ_score>=20:
    return 20
  elif univ_score>=10:
    return 10
  else:
    return 0

In [ ]:
a = 0
b = 1

# apply_matrix

In [ ]:
#학습, 검증 분리
train, test = [], []
apply_train_groupby = apply_train_df.groupby('resume_seq')['recruitment_seq'].apply(list)
for uid, iids in zip(apply_train_groupby.index.tolist(), apply_train_groupby.values.tolist()):
    for iid in iids[:-1]:
        train.append([uid,iid])
    test.append([uid, iids[-1]])

In [ ]:
apply_train = pd.DataFrame(train, columns=['resume_seq', 'recruitment_seq'])
apply_test = pd.DataFrame(test, columns=['resume_seq', 'recruitment_seq'])
apply_pred = apply_train_df.copy()

In [ ]:
# 사용자-아이템 행렬 생성: 구직자가 해당 채용 공고에 지원했으면 1, 아니면 0으로 설정
apply_train_item_matrix = apply_train.groupby(['resume_seq', 'recruitment_seq']).size().unstack(fill_value=0)
apply_train_similarity = cosine_similarity(apply_train_item_matrix) # 사용자 간의 유사성 계산

apply_pred_item_matrix = apply_pred.groupby(['resume_seq', 'recruitment_seq']).size().unstack(fill_value=0)
apply_pred_similarity = cosine_similarity(apply_pred_item_matrix) # 사용자 간의 유사성 계산

In [ ]:
apply_train_matrix = pd.DataFrame(apply_train_similarity) # DataFrame 생성
apply_pred_matrix = pd.DataFrame(apply_pred_similarity) # DataFrame 생성

# 행과 열 이름 설정
apply_train_matrix.columns = [f'U{i:05d}' for i in range(1, 8483)]
apply_train_matrix.index = [f'U{i:05d}' for i in range(1, 8483)]

apply_pred_matrix.columns = [f'U{i:05d}' for i in range(1, 8483)]
apply_pred_matrix.index = [f'U{i:05d}' for i in range(1, 8483)]

# resume_matrix

## resume_matrix_career_job_code null 없애기

In [ ]:
resume_train_df['job_code_seq1'].value_counts()

재료·화학·섬유·의복    7288
디자인             536
경영·기획·회계·사무     489
영업·판매·TM        158
건설·기계·전기·전자       5
전문·교육·자격          3
기타 직종             1
IT·게임             1
문화·예술·신문·방송       1
Name: job_code_seq1, dtype: int64

In [ ]:
resume_train_df['career_job_code'].value_counts()

섬유;봉제;가방;의류            5059
도소매;유통;무역               821
디자인;CAD;설계              653
쇼핑몰;전자상거래;경매            133
기타 서비스                  119
학교;학원;직원훈련(교육서비스)        99
기타 제조                    81
광고;홍보                    81
영화;방송;공연;문화;스포츠          79
생활용품;화장품;완구              53
건설;토목;건축                 52
식음료;식품가공                 49
금융;보험;증권                 43
호텔;숙박;관광;여행;외식           35
전기;전자;통신장비               35
출판;인쇄;편집;영상              30
신문;서적;출판;인쇄              28
자동차;운송장비;조선;항공           27
공공행정;경찰;기관;협회            26
의료;제약;건강                 22
인터넷서비스;포털;컨텐츠            20
컴퓨터;사무기기;소비재             19
운송;운수;통신                 15
철강;금속;비금속;재료             15
반도체;LCD;광학;정밀            13
화학;고무;플라스틱               13
목재;나무;종이;가구              11
네트워크;통신;텔레콤              11
기타 공공;개인서비스              10
병원;제약;사회복지               10
기계;각종 장비                  9
웹에이전시                     9
인력공급;시설유지;TM(용역;도급)       8
IT 컨설팅;조사분석;통계            8
게임;캐릭터;애니메이션              7
전기;가스;수도;에너지        

In [ ]:
resume_train_df['degree'].value_counts()

4    7453
3     463
5     349
2     211
6       6
Name: degree, dtype: int64

In [ ]:
resume_train_df['graduate_date'].value_counts()

0       792
2009    542
2006    516
2008    506
2007    494
2010    485
2011    474
2012    444
2004    408
2013    401
2014    385
2005    372
2003    323
2002    302
2001    278
2015    273
2016    204
2000    202
1999    167
2017    122
1998    118
1997    111
1996     93
1995     76
1994     63
2018     58
1992     56
1993     45
1991     32
2019     24
1990     21
1989     20
1988     16
1987     13
1986     10
2020      9
1985      6
1984      3
1979      3
2021      3
1983      2
1980      2
1982      2
1970      1
1977      1
1978      1
1973      1
1974      1
2022      1
Name: graduate_date, dtype: int64

In [ ]:
resume_train_df['updated_date'].value_counts()

2020-04-08    43
2020-04-06    41
2020-02-25    40
2020-11-13    40
2020-01-14    40
              ..
2020-06-06    10
2020-01-01     8
2020-10-03     8
2020-12-25     7
2020-02-29     4
Name: updated_date, Length: 366, dtype: int64

In [ ]:
resume_train_df['career_month'].value_counts()

0      1095
36      258
12      237
24      230
48      215
       ... 
262       1
294       1
399       1
432       1
245       1
Name: career_month, Length: 295, dtype: int64

In [ ]:
resume_train_df[(resume_train_df['job_code_seq1']=='재료·화학·섬유·의복')
&(resume_train_df['career_job_code'] == '섬유;봉제;가방;의류')
&(resume_train_df['degree'] == 4)
& (resume_train_df['graduate_date']==2011)]['hope_salary'].value_counts()

0.0       149
2300.0     13
2500.0     12
2700.0     11
3700.0     11
2900.0      9
2100.0      8
3500.0      8
3100.0      6
4100.0      5
3300.0      5
1900.0      4
4700.0      3
4500.0      3
1700.0      2
4300.0      2
3900.0      2
5750.0      1
6750.0      1
1300.0      1
1500.0      1
Name: hope_salary, dtype: int64

In [ ]:
resume_train = resume_train_df.copy()
resume_train = resume_train[['resume_seq', 'degree', 'job_code_seq1', 'updated_date','hope_salary', 'last_salary', 'career_month','text_keyword', 'career_job_code','graduate_date']] # 여기 수정

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# 'career_job_code'가 NaN이면서 'text_keyword'에 '상품기획'이 있는 행 선택
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('상품기획'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '섬유;봉제;가방;의류'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('MD'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '섬유;봉제;가방;의류'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('스타일디자이너'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '섬유;봉제;가방;의류'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('일러스트레이터'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '섬유;봉제;가방;의류'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('영업기획'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '도소매;유통;무역'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('골프웨어디자이너'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '섬유;봉제;가방;의류'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('컬러리스트자격증'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '섬유;봉제;가방;의류'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('머천다이저'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '도소매;유통;무역'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('패션'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '디자인;CAD;설계'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('피팅'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '디자인;CAD;설계'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('화학'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '화학;고무;플라스틱'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('캐주얼'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '섬유;봉제;가방;의류'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('영업'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '도소매;유통;무역'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('디자이너'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '섬유;봉제;가방;의류'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('디자인'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '디자인;CAD;설계'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('마케팅'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '섬유;봉제;가방;의류'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('경영'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '도소매;유통;무역'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('여성'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '섬유;봉제;가방;의류'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('남성'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '섬유;봉제;가방;의류'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('구매'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '도소매;유통;무역'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('인사'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '도소매;유통;무역'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('디스플레이'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '반도체;LCD;광학;정밀'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('사무'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '도소매;유통;무역'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('생산'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '섬유;봉제;가방;의류'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('슈퍼바이저'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '신문;서적;출판;인쇄'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('중국'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '도소매;유통;무역'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('일러스트'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '도소매;유통;무역'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('일본어'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '섬유;봉제;가방;의류'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('팀'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '섬유;봉제;가방;의류'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('삽마스터'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '섬유;봉제;가방;의류'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('총무'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '도소매;유통;무역'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('매장관리'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '도소매;유통;무역'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('성실한인재'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '섬유;봉제;가방;의류'
resume_train.loc[resume_train['career_job_code'].isnull(), 'career_job_code'] = '섬유;봉제;가방;의류'

## resume_text

In [ ]:
resume_train_text = resume_train_df[['resume_seq', 'text_keyword']]

In [ ]:
resume_train_text.fillna('무응답', inplace=True)

In [ ]:
# ';'로 구분된 값을 분리하여 새로운 행 생성
new_rows = []
for index, row in resume_train_text.iterrows():
    keywords = row['text_keyword'].split(';')
    for keyword in keywords:
        new_rows.append({'resume_seq': row['resume_seq'], 'text_keyword': keyword})

# 새로운 행을 데이터프레임에 추가
resume_train_text = pd.DataFrame(new_rows)

## resume_matrix 범주형 데이터 형변환

In [ ]:
no_category_cols = ['hope_salary','last_salary', 'career_month', 'graduate_date']
category_cols = ['degree']

In [ ]:
resume_train_no_category = resume_train[no_category_cols]
resume_train_category = resume_train[category_cols]

In [ ]:
aaa = resume_train_df.copy() # 1에 너무 양이 몰려서 2에서 가져오는건데 데이터를 변경하는거라 좋지 않음
aaa['job_code_seq1'].value_counts() # 희망 직무가 같은게 7288인걸 보면 흐으으으으으으음
aaa.loc[(aaa['job_code_seq1'] == '재료·화학·섬유·의복') & (aaa['job_code_seq2'] != ''), 'job_code_seq1'] = aaa.loc[(aaa['job_code_seq1'] == '재료·화학·섬유·의복') & (aaa['job_code_seq2'] != ''), 'job_code_seq2']
aaa['job_code_seq1'].fillna('재료·화학·섬유·의복', inplace=True)
resume_train['job_code_seq1'] = aaa['job_code_seq1']

In [ ]:
resume_train['month'] = resume_train['updated_date'].apply(lambda x : int(x[5:7]))
resume_train['day'] = resume_train['updated_date'].apply(lambda x : int(x[8:10]))

In [ ]:
resume_train = resume_train.drop(columns=['updated_date'])
resume_train = resume_train.drop(columns=['text_keyword'])

In [ ]:
resume_train['degree'] = resume_train['degree'].astype("category") # 원핫 인코딩 할 것들(숫자)

In [ ]:
_train_encoded = pd.DataFrame()

for col in category_cols: # 수정
  _encoder = ce.OneHotEncoder(use_cat_names=True)
  _encoder.fit(resume_train[col])
  _encoded = _encoder.transform(resume_train[col])
  _train_encoded = pd.concat([_train_encoded, _encoded], axis=1)

In [ ]:
resume_tmp = resume_train[['resume_seq']]
resume_tmp = pd.merge(resume_tmp, _train_encoded, left_index=True, right_index=True, how='inner')

## 수치형 스케일링

In [ ]:
# 스케일링 적용할 컬럼 선언
scaling_cols = ['hope_salary','last_salary', 'career_month', 'graduate_date']

# 수치형 데이터 프레임에서 스케일링 적용할 컬럼이 포함된 데이터 프레임 생성
resume_train_scaling = resume_train[scaling_cols]

print(f'{resume_train_scaling.shape}')
resume_train_scaling.head(3)

(8482, 4)


,hope_salary,last_salary,career_month,graduate_date
0,3500.0,3500.0,67,2008
1,0.0,3700.0,84,0
2,3500.0,3100.0,121,1979


In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# 인스턴스 생성
std = StandardScaler()
# 학습
std.fit(resume_train_scaling)

StandardScaler()

In [ ]:
# 반영
resume_train_scaled = std.transform(resume_train_scaling)

In [ ]:
resume_train_scaled.shape

(8482, 4)

In [ ]:
resume_train_scaling.shape

(8482, 4)

In [ ]:
resume_train_scaling

,hope_salary,last_salary,career_month,graduate_date
0,3500.0,3500.0,67,2008
1,0.0,3700.0,84,0
2,3500.0,3100.0,121,1979
3,0.0,2500.0,24,2012
4,1900.0,0.0,0,2010
...,...,...,...,...
8477,0.0,6250.0,204,1994
8478,2300.0,1900.0,12,2013
8479,0.0,0.0,0,2008
8480,0.0,0.0,18,2016


In [ ]:
resume_train_scaled

array([[ 1.12812625,  0.49468233, -0.03791036,  0.32264053],
       [-0.74770321,  0.59418659,  0.22961995, -3.11586403],
       [ 1.12812625,  0.29567382,  0.81189179,  0.27298085],
       ...,
       [-0.74770321, -1.24664218, -1.09229452,  0.32264053],
       [-0.74770321, -1.24664218, -0.80902713,  0.33633975],
       [ 0.37779447, -0.74912089, -1.04508329,  0.32435293]])

In [ ]:
df = pd.DataFrame(resume_train_scaling)

In [ ]:
df[['scaled_1', 'scaled_2', 'scaled_3', 'scaled_4']] = resume_train_scaled
df

,hope_salary,last_salary,career_month,graduate_date,scaled_1,scaled_2,scaled_3,scaled_4
0,3500.0,3500.0,67,2008,1.128126,0.494682,-0.037910,0.322641
1,0.0,3700.0,84,0,-0.747703,0.594187,0.229620,-3.115864
2,3500.0,3100.0,121,1979,1.128126,0.295674,0.811892,0.272981
3,0.0,2500.0,24,2012,-0.747703,-0.002839,-0.714605,0.329490
4,1900.0,0.0,0,2010,0.270604,-1.246642,-1.092295,0.326065
...,...,...,...,...,...,...,...,...
8477,0.0,6250.0,204,1994,-0.747703,1.862866,2.118069,0.298667
8478,2300.0,1900.0,12,2013,0.484985,-0.301352,-0.903450,0.331203
8479,0.0,0.0,0,2008,-0.747703,-1.246642,-1.092295,0.322641
8480,0.0,0.0,18,2016,-0.747703,-1.246642,-0.809027,0.336340


## 수치형 범주형 합치기

In [ ]:
no_category_cols = ['hope_salary','last_salary', 'career_month', 'graduate_date']

In [ ]:
resume_item_matrix = resume_train.groupby(['resume_seq', 'job_code_seq1']).size().unstack(fill_value=0)
resume_item_matrix2 = resume_train.groupby(['resume_seq', 'career_job_code']).size().unstack(fill_value=0)
resume_text_matrix = resume_train_text.groupby(['resume_seq', 'text_keyword']).size().unstack(fill_value=0)

resume_item_matrix = pd.merge(resume_item_matrix, resume_tmp, on='resume_seq', how='left')
resume_item_matrix = pd.merge(resume_item_matrix, resume_item_matrix2, on='resume_seq', how='left')
resume_item_matrix = pd.merge(resume_item_matrix, resume_text_matrix, on='resume_seq', how='left')

resume_item_matrix.set_index('resume_seq', inplace=True)
resume_similarity = cosine_similarity(resume_item_matrix) # 사용자 간의 유사성 계산

In [ ]:
for col in no_category_cols:
  resume_tmp_item = resume_train[['resume_seq', col]]
  resume_item_matrix2 = resume_tmp_item.groupby(['resume_seq', col]).size().unstack(fill_value=0)
  resume_item_matrix = pd.merge(resume_item_matrix, resume_item_matrix2, on='resume_seq', how='left')

In [ ]:
resume_item_matrix

,IT·게임,건설·기계·전기·전자,경영·기획·회계·사무,기술·과학·산업,기타 직종,디자인_x,문화·예술·신문·방송,영업·판매·TM,재료·화학·섬유·의복,전문·교육·자격,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
resume_seq,,,,,,,,,,,,,,,,,,,,,
U00001,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
U00002,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
U00003,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
U00004,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
U00005,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
U08478,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
U08479,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
U08480,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
resume_matrix = pd.DataFrame(resume_similarity) # DataFrame 생성
# 행과 열 이름 설정
resume_matrix.columns = [f'U{i:05d}' for i in range(1, 8483)]
resume_matrix.index = [f'U{i:05d}' for i in range(1, 8483)]

# certificate

In [ ]:
certificate_df['certificate_contents'].value_counts()

자동차 운전면허 1종보통       563
운전면허2종보통            547
자동차운전면허 1종 보통       453
자동차 운전면허 (2종 보통)    441
자동차운전면허1종 보통        359
                   ... 
보육교사3급자격증             1
컴퓨터활용능력평가             1
합기도3단 유도2단            1
휴넷 마케팅 MBA            1
수치제어밀링기능사             1
Name: certificate_contents, Length: 1786, dtype: int64

In [ ]:
tmp = apply_train_df.copy()
tmp = tmp.groupby('resume_seq')['recruitment_seq'].apply(';'.join).reset_index()
tmp = tmp.drop(columns = ['recruitment_seq'])

In [ ]:
# NaN 값을 빈 문자열로 대체하고, 문자열로 형변환
certificate_df['certificate_contents'] = certificate_df['certificate_contents'].fillna('').astype(str)

# 자동차 운전면허와 관련된 내용을 필터링
certificate_df = certificate_df[~certificate_df['certificate_contents'].str.contains('자동차 운전면허')]
certificate_df = certificate_df[~certificate_df['certificate_contents'].str.contains('운전')]
certificate_df = certificate_df[~certificate_df['certificate_contents'].str.contains('면허')]
'''certificate_df = certificate_df[~certificate_df['certificate_contents'].str.contains('워드')]
certificate_df = certificate_df[~certificate_df['certificate_contents'].str.contains('유통')]
certificate_df = certificate_df[~certificate_df['certificate_contents'].str.contains('샵')]
certificate_df = certificate_df[~certificate_df['certificate_contents'].str.contains('활')]'''
# 1인 건을 필터링하여 새로운 데이터프레임 생성
certificate_df = certificate_df[certificate_df.groupby('certificate_contents')['certificate_contents'].transform('count') > 1]
certificate_df = certificate_df[certificate_df.groupby('certificate_contents')['certificate_contents'].transform('count') < 200]
# 필터링된 결과를 확인
print(certificate_df['certificate_contents'].value_counts())

컬러리스트 산업기사     167
샵마스터 3급        141
컬러리스트 기사       134
GTQ 포토샵 1급     125
패션머천다이징산업기사    114
              ... 
품질관리기사           2
GTQ 일러스트 3급      2
메이크업             2
6시그마 GB          2
레크레이션1급          2
Name: certificate_contents, Length: 699, dtype: int64


In [ ]:
certificate_item_matrix = certificate_df.groupby(['resume_seq', 'certificate_contents']).size().unstack(fill_value=0)
certificate_item_matrix = pd.merge(tmp, certificate_item_matrix, on='resume_seq', how='left')
certificate_item_matrix.fillna(0, inplace=True)
certificate_item_matrix.set_index('resume_seq', inplace=True)
certificate_similarity = cosine_similarity(certificate_item_matrix) # 사용자 간의 유사성 계산

In [ ]:
certificate_matrix = pd.DataFrame(certificate_similarity) # DataFrame 생성
# 행과 열 이름 설정
certificate_matrix.columns = [f'U{i:05d}' for i in range(1, 8483)]
certificate_matrix.index = [f'U{i:05d}' for i in range(1, 8483)]

# education_matrix

In [ ]:
education_train_df

,resume_seq,hischool_type_seq,hischool_special_type,hischool_nation,hischool_gender,hischool_location_seq,univ_type_seq1,univ_type_seq2,univ_transfer,univ_location,univ_major,univ_sub_major,univ_major_type,univ_score
0,U01419,21,일반고,사립,남자학교,3,5,5,0,3,NaN,NaN,9,60.0
1,U03375,21,일반고,사립,여자학교,3,5,5,0,3,NaN,NaN,4,80.0
2,U06523,21,일반고,사립,남여공학,3,5,5,0,3,NaN,NaN,8,70.0
3,U06619,21,일반고,사립,남여공학,5,5,5,0,5,NaN,NaN,8,80.0
4,U05015,16,특성화고,공립,남여공학,3,5,5,0,3,NaN,NaN,9,80.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8477,U06326,21,일반고,사립,여자학교,9,5,5,0,7,섬우패션학부의류패션전공,NaN,10,70.0
8478,U07284,0,기타,기타,기타,0,5,5,0,11,NaN,NaN,3,70.0
8479,U04109,21,일반고,공립,남여공학,5,0,0,0,0,NaN,NaN,9,80.0
8480,U01863,21,일반고,사립,여자학교,15,3,5,0,15,의류학과생활과학계열,NaN,9,80.0


In [ ]:
education_train_df['hischool_special_type'].value_counts()

일반고     5820
기타      1724
자율고      501
특성화고     281
특목고      156
Name: hischool_special_type, dtype: int64

In [ ]:
education_train_df['hischool_type_seq'].value_counts()

21    6277
0     1724
16     147
8      112
19      92
20      51
22      36
2       17
18      17
23       5
12       3
10       1
Name: hischool_type_seq, dtype: int64

In [ ]:
education_train_df['hischool_gender'].value_counts()

남여공학    2607
여자학교    2423
남자학교    1728
기타      1724
Name: hischool_gender, dtype: int64

In [ ]:
education_train_df['univ_type_seq1'].value_counts()

5    6455
3    1006
0     923
4      83
2      13
6       2
Name: univ_type_seq1, dtype: int64

In [ ]:
education_train_df['univ_major_type'].value_counts()

9     3255
3     1028
4      863
19     714
5      604
0      566
8      424
10     263
20     136
2       89
18      88
7       86
13      62
6       55
14      48
17      41
23      32
11      32
15      29
16      24
12      21
22      14
21       8
Name: univ_major_type, dtype: int64

In [ ]:
education_train_df['univ_score'].value_counts()

80.0    3550
70.0    2538
90.0    1107
60.0     507
0.0      337
        ... 
1.9        1
81.8       1
51.3       1
4.1        1
74.0       1
Name: univ_score, Length: 96, dtype: int64

In [ ]:
education_train_df = education_train_df[['resume_seq', 'hischool_special_type', 'hischool_type_seq',  'hischool_gender', 'univ_type_seq1','univ_major_type', 'univ_score']]

In [ ]:
education_train_df['sub_univ_score'] = education_train_df['univ_score'].map(lambda x: check_univ_score(x))

education_train_df = education_train_df.drop(columns=['univ_score'])

In [ ]:
education_train_df['hischool_type_seq'] = education_train_df['hischool_type_seq'].astype("category")# 원핫 인코딩 할 것들(숫자)
education_train_df['hischool_gender'] = education_train_df['hischool_gender'].astype("category")
education_train_df['univ_type_seq1'] = education_train_df['univ_type_seq1'].astype("category")
education_train_df['univ_major_type'] = education_train_df['univ_major_type'].astype("category")
education_train_df['sub_univ_score'] = education_train_df['sub_univ_score'].astype("category")

In [ ]:
_train_encoded = pd.DataFrame()

for col in ['hischool_type_seq',  'hischool_gender', 'univ_type_seq1','univ_major_type', 'sub_univ_score']: # 수정
  _encoder = ce.OneHotEncoder(use_cat_names=True)
  _encoder.fit(education_train_df[col])
  _encoded = _encoder.transform(education_train_df[col])
  _train_encoded = pd.concat([_train_encoded, _encoded], axis=1)

In [ ]:
education_tmp = education_train_df[['resume_seq']]
education_tmp = pd.merge(education_tmp, _train_encoded, left_index=True, right_index=True, how='inner')

In [ ]:
education_item_matrix = education_train_df.groupby(['resume_seq', 'hischool_special_type']).size().unstack(fill_value=0)
education_item_matrix = pd.merge(education_item_matrix, education_tmp, on='resume_seq', how='left')

In [ ]:
education_item_matrix.set_index('resume_seq', inplace=True)
education_similarity = cosine_similarity(education_item_matrix) # 사용자 간의 유사성 계산

In [213]:
education_item_matrix

,기타,일반고,자율고,특목고,특성화고,hischool_type_seq_21.0,hischool_type_seq_16.0,hischool_type_seq_2.0,hischool_type_seq_0.0,hischool_type_seq_8.0,...,sub_univ_score_60.0,sub_univ_score_80.0,sub_univ_score_70.0,sub_univ_score_90.0,sub_univ_score_20.0,sub_univ_score_0.0,sub_univ_score_50.0,sub_univ_score_10.0,sub_univ_score_30.0,sub_univ_score_40.0
resume_seq,,,,,,,,,,,,,,,,,,,,,
U00001,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
U00002,1,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
U00003,0,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
U00004,0,1,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
U00005,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
U08478,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
U08479,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
U08480,1,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0


# language_matrix

In [227]:
language_train_df = language_train_df[['resume_seq', 'exam_name']]

In [228]:
tmp = apply_train_df.copy()
tmp = tmp.groupby('resume_seq')['recruitment_seq'].apply(';'.join).reset_index()
tmp = tmp.drop(columns = ['recruitment_seq'])

In [229]:
language_train_item_matrix = language_train_df.groupby(['resume_seq', 'exam_name']).size().unstack(fill_value=0)

In [230]:
language_train_item_matrix = pd.merge(tmp, language_train_item_matrix, on='resume_seq', how='left').fillna(0)
language_train_item_matrix.set_index('resume_seq', inplace=True)

# 회사 최소 조건

In [ ]:
resume_c = resume_train_df.copy()
recruitment_c = recruitment_train_df.copy()

In [ ]:
# 'resume_seq' 열을 기준으로 정렬
resume_c = resume_c.sort_values(by='resume_seq')
# 인덱스를 순차적으로 변경
resume_c['resume_seq'] = [f'U{i:05}' for i in range(1, len(resume_c) + 1)]
resume_c = resume_c.set_index('resume_seq')

# 'resume_seq' 열을 기준으로 정렬
recruitment_c = recruitment_c.sort_values(by='recruitment_seq')
# 인덱스를 순차적으로 변경
recruitment_c['recruitment_seq'] = [f'R{i:05}' for i in range(1, len(recruitment_c) + 1)]
recruitment_c = recruitment_c.set_index('recruitment_seq')

In [ ]:
resume_c = resume_c[['degree']]
recruitment_c = recruitment_c[['education']]

In [ ]:
apply_item_matrix_new = []
count = 0

tem_6 = [] # U05886이 6학년
tem_5 = [] # U08422이 5학년
tem_4 = [] # U04600이 4학년
tem_3 = [] # U03400이 3학년
tem_2 = [] # U04220이 2학년

for recruitment_seq in range(len(recruitment_c)): #
  if resume_c.loc['U05886']['degree'] < recruitment_c.iloc[recruitment_seq]['education']:
        yy = False
  else:
        yy = True
  tem_6.append(yy)

for recruitment_seq in range(len(recruitment_c)):
  if resume_c.loc['U08422']['degree'] < recruitment_c.iloc[recruitment_seq]['education']:
        yy = False
  else:
        yy = True
  tem_5.append(yy)

for recruitment_seq in range(len(recruitment_c)):
  if resume_c.loc['U04600']['degree'] < recruitment_c.iloc[recruitment_seq]['education']:
        yy = False
  else:
        yy = True
  tem_4.append(yy)

for recruitment_seq in range(len(recruitment_c)):
  if resume_c.loc['U03400']['degree'] < recruitment_c.iloc[recruitment_seq]['education']:
        yy = False
  else:
        yy = True
  tem_3.append(yy)

for recruitment_seq in range(len(recruitment_c)):
  if resume_c.loc['U04220']['degree'] < recruitment_c.iloc[recruitment_seq]['education']:
        yy = False
  else:
        yy = True
  tem_2.append(yy)

# apply_item_matrix의 인덱스(=resume_seq)와 컬럼(=recruitment_seq)을 뽑아냄
for resume_seq in range(len(resume_c)):
    if resume_c.iloc[resume_seq]['degree'] == 6:
      apply_item_matrix_new.append(tem_6)
    elif resume_c.iloc[resume_seq]['degree'] == 5:
      apply_item_matrix_new.append(tem_5)
    elif resume_c.iloc[resume_seq]['degree'] == 4:
      apply_item_matrix_new.append(tem_4)
    elif resume_c.iloc[resume_seq]['degree'] == 3:
      apply_item_matrix_new.append(tem_3)
    else:
      apply_item_matrix_new.append(tem_2)

# 새로운 DataFrame 생성
apply_item_matrix_new = pd.DataFrame(apply_item_matrix_new, index=apply_pred_item_matrix.index, columns=apply_pred_item_matrix.columns)
apply_item_matrix_new

recruitment_seq,R00001,R00002,R00003,R00004,R00005,R00006,R00007,R00008,R00009,R00010,...,R06686,R06687,R06688,R06689,R06690,R06691,R06692,R06693,R06694,R06695
resume_seq,,,,,,,,,,,,,,,,,,,,,
U00001,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
U00002,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
U00003,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
U00004,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
U00005,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
U08478,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
U08479,True,True,True,True,True,False,True,True,True,True,...,True,True,True,True,True,True,True,False,False,True
U08480,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


# train 결과 도출

In [185]:
resume_all_matrix = pd.merge(resume_item_matrix, certificate_item_matrix, on='resume_seq', how='left')
resume_all_matrix = pd.merge(resume_all_matrix, education_item_matrix, on='resume_seq', how='left')
resume_all_matrix = pd.merge(resume_all_matrix, language_train_item_matrix, on='resume_seq', how='left')

In [186]:
all_similarity = cosine_similarity(resume_all_matrix)

In [220]:
# train_사용자 간의 유사성 계산
user_train_similarity = apply_train_similarity * all_similarity
user_predicted_scores = user_train_similarity.dot(apply_train_item_matrix) / np.array([np.abs(user_train_similarity).sum(axis=1)]).T

In [221]:
item_train_similarity = cosine_similarity(apply_train_item_matrix.T)
item_train_predicted_scores = apply_train_item_matrix.dot(item_train_similarity)

In [222]:
user_matrix = pd.DataFrame(user_train_similarity) # DataFrame 생성
# 행과 열 이름 설정
user_matrix.columns = [f'U{i:05d}' for i in range(1, 8483)]
user_matrix.index = [f'U{i:05d}' for i in range(1, 8483)]

In [223]:
sort_user_list = []
applied_jobs = []
alpha = 0

for idx, user in enumerate(user_matrix.index):
    # 해당 사용자가 지원한 채용 공고
    applied_job = apply_train[apply_train['resume_seq'] == user]['recruitment_seq'].values
    applied_jobs.append([user, applied_job])

In [224]:
# 이미 지원한 채용 공고 제외하고 추천
recommendations = []


for idx, user in enumerate(apply_train_item_matrix.index):
  re_applied_jobs = []
  # 해당 사용자의 추천 점수 (높은 점수부터 정렬)
  sorted_job_indices = (user_predicted_scores[idx] + item_train_predicted_scores.loc[user]*alpha).argsort()[::-1]
  for idxx, j in enumerate(sorted_job_indices):
    if(len(re_applied_jobs) >= 5):
      break
    else:
      job = apply_train_item_matrix.columns[j]
      if (job not in applied_jobs[idx][1] and apply_item_matrix_new.loc[user].loc[job] and (job not in re_applied_jobs)):
        re_applied_jobs.append(job)
        recommendations.append([user, job])

In [225]:
val_prediction = pd.DataFrame(recommendations, columns=['resume_seq', 'recruitment_seq'])

In [226]:
recall5(apply_test,val_prediction)

0.0010610705022400376

# pred 결과 도출

In [ ]:
pred_resume_all_matrix = pd.merge(resume_item_matrix, certificate_item_matrix, on='resume_seq', how='left')
pred_resume_all_matrix = pd.merge(resume_all_matrix, education_item_matrix, on='resume_seq', how='left')
pred_resume_all_matrix = pd.merge(resume_all_matrix, language_train_item_matrix, on='resume_seq', how='left')

In [ ]:
pred_all_similarity = cosine_similarity(pred_resume_all_matrix)

In [ ]:
# train_사용자 간의 유사성 계산
user_similarity = a*apply_pred_similarity + b*pred_all_similarity
pred_user_predicted_scores = user_similarity.dot(apply_pred_item_matrix) / np.array([np.abs(user_similarity).sum(axis=1)]).T

In [ ]:
pred_item_train_similarity = cosine_similarity(apply_pred_item_matrix.T)
pred_item_train_predicted_scores = apply_pred_item_matrix.dot(pred_item_train_similarity)

In [ ]:
user_matrix = pd.DataFrame(user_similarity) # DataFrame 생성
# 행과 열 이름 설정
user_matrix.columns = [f'U{i:05d}' for i in range(1, 8483)]
user_matrix.index = [f'U{i:05d}' for i in range(1, 8483)]

In [ ]:
sort_user_list = []
applied_jobs = []
alpha = 0.032

for idx, user in enumerate(user_matrix.index):
    # 해당 사용자가 지원한 채용 공고
    applied_job = apply_pred[apply_pred['resume_seq'] == user]['recruitment_seq'].values
    applied_jobs.append([user, applied_job])

In [ ]:
# 이미 지원한 채용 공고 제외하고 추천
recommendations = []

for idx, user in enumerate(apply_pred_item_matrix.index):
  re_applied_jobs = []
  # 해당 사용자의 추천 점수 (높은 점수부터 정렬)
  sorted_job_indices = (pred_user_predicted_scores[idx] + pred_item_train_predicted_scores.loc[user]*alpha).argsort()[::-1]
  for idxx, j in enumerate(sorted_job_indices):
    if(len(re_applied_jobs) >= 5):
      break
    else:
      job = apply_pred_item_matrix.columns[j]
      if (job not in applied_jobs[idx][1] and apply_item_matrix_new.loc[user].loc[job] and (job not in re_applied_jobs)):
        re_applied_jobs.append(job)
        recommendations.append([user, job])

In [ ]:
# sample_submission.csv 형태로 DataFrame 생성
top_recommendations = pd.DataFrame(recommendations, columns=['resume_seq', 'recruitment_seq'])

top_recommendations.to_csv('./H_submit.csv', index=False)